<a href="https://colab.research.google.com/github/louispaulet/Data_dependencies_and_data_integration_course/blob/main/DDDI_Project_UCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install pandas-profiling

In [ ]:
import pandas as pd
import io

df = pd.read_csv('Table1.csv')
df


,A,B,C,D,E
0,0,0,0,2,0
1,0,1,0,2,0
2,0,2,0,2,2
3,0,3,1,2,0
4,4,1,1,1,4
5,4,3,1,1,2
6,0,0,1,2,0


In [ ]:
df.describe()

,A,B,C,D,E
count,7.000000,7.000000,7.000000,7.000000,7.000000
mean,1.142857,1.428571,0.571429,1.714286,1.142857
std,1.951800,1.272418,0.534522,0.487950,1.573592
min,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.500000,0.000000,1.500000,0.000000
50%,0.000000,1.000000,1.000000,2.000000,0.000000
75%,2.000000,2.500000,1.000000,2.000000,2.000000
max,4.000000,3.000000,1.000000,2.000000,4.000000


We start easy with a ucc_2() function that only checks pairs of columns

In [ ]:
def ucc_2(df):
  list_of_uccs = []
  #for every column
  for col in df:
    #for every combination of column
    for col2 in df:
      #if the columns are not the same one and they do not have duplicate rows when joined row by row:
      if (col != col2 and len(df[col].index) == len(((pd.concat([df2[col], df2[col2]], axis=1)).drop_duplicates()).index)):
        #add to the list of uccs
        list_of_uccs.append((col, col2))
  return list_of_uccs

#we run ucc() on default dataframe and get an empty result because there are no UCCs  
ucc_2(df)        

[]

In [ ]:
#create a dataframe with an "index" column that will make supersets of "test_col" all UCCs
df2 = df.copy()
df2['test_col'] = range(1, len(df) + 1)
df2

,A,B,C,D,E,test_col
0,0,0,0,2,0,1
1,0,1,0,2,0,2
2,0,2,0,2,2,3
3,0,3,1,2,0,4
4,4,1,1,1,4,5
5,4,3,1,1,2,6
6,0,0,1,2,0,7


In [ ]:
#we can see that the function returns supersets of "test_col" as UCCs -- should we keep the symetry ?
ucc_2(df2)

[('A', 'test_col'),
 ('B', 'test_col'),
 ('C', 'test_col'),
 ('D', 'test_col'),
 ('E', 'test_col'),
 ('test_col', 'A'),
 ('test_col', 'B'),
 ('test_col', 'C'),
 ('test_col', 'D'),
 ('test_col', 'E')]

We need to find all combinations of columns that are UCC, not only two columns.
We use itertools to generate all combinations of columns from df :

In [ ]:
from itertools import combinations 

column_combination = []
for i in range(1, len(df.columns)+1):
  for item in list(combinations(df.columns, i)):
    column_combination.append(item)
column_combination

[('A',),
 ('B',),
 ('C',),
 ('D',),
 ('E',),
 ('A', 'B'),
 ('A', 'C'),
 ('A', 'D'),
 ('A', 'E'),
 ('B', 'C'),
 ('B', 'D'),
 ('B', 'E'),
 ('C', 'D'),
 ('C', 'E'),
 ('D', 'E'),
 ('A', 'B', 'C'),
 ('A', 'B', 'D'),
 ('A', 'B', 'E'),
 ('A', 'C', 'D'),
 ('A', 'C', 'E'),
 ('A', 'D', 'E'),
 ('B', 'C', 'D'),
 ('B', 'C', 'E'),
 ('B', 'D', 'E'),
 ('C', 'D', 'E'),
 ('A', 'B', 'C', 'D'),
 ('A', 'B', 'C', 'E'),
 ('A', 'B', 'D', 'E'),
 ('A', 'C', 'D', 'E'),
 ('B', 'C', 'D', 'E'),
 ('A', 'B', 'C', 'D', 'E')]

This ucc() function manages all combinations of columns

In [ ]:
def ucc(df):

  column_combination = []
  for i in range(1, len(df.columns)+1):
    for item in list(combinations(df.columns, i)):
      column_combination.append(item)

  list_of_uccs = []
  for comb in column_combination:
    frames = []
    for col_name in list(comb):
      frames.append(df[col_name])
    # print(pd.concat(frames, axis=1).drop_duplicates())
    if (len(frames[0].index) == len(((pd.concat(frames, axis=1)).drop_duplicates()).index)):
      list_of_uccs.append(comb)


  return list_of_uccs

ucc(df)

[('A', 'B', 'C'),
 ('B', 'C', 'D'),
 ('B', 'C', 'E'),
 ('A', 'B', 'C', 'D'),
 ('A', 'B', 'C', 'E'),
 ('B', 'C', 'D', 'E'),
 ('A', 'B', 'C', 'D', 'E')]

Now we try it with the df2 that contains "test_col" to see if all supersets of "test_col" are generated :

In [ ]:
ucc(df2)

[('test_col',),
 ('A', 'test_col'),
 ('B', 'test_col'),
 ('C', 'test_col'),
 ('D', 'test_col'),
 ('E', 'test_col'),
 ('A', 'B', 'C'),
 ('A', 'B', 'test_col'),
 ('A', 'C', 'test_col'),
 ('A', 'D', 'test_col'),
 ('A', 'E', 'test_col'),
 ('B', 'C', 'D'),
 ('B', 'C', 'E'),
 ('B', 'C', 'test_col'),
 ('B', 'D', 'test_col'),
 ('B', 'E', 'test_col'),
 ('C', 'D', 'test_col'),
 ('C', 'E', 'test_col'),
 ('D', 'E', 'test_col'),
 ('A', 'B', 'C', 'D'),
 ('A', 'B', 'C', 'E'),
 ('A', 'B', 'C', 'test_col'),
 ('A', 'B', 'D', 'test_col'),
 ('A', 'B', 'E', 'test_col'),
 ('A', 'C', 'D', 'test_col'),
 ('A', 'C', 'E', 'test_col'),
 ('A', 'D', 'E', 'test_col'),
 ('B', 'C', 'D', 'E'),
 ('B', 'C', 'D', 'test_col'),
 ('B', 'C', 'E', 'test_col'),
 ('B', 'D', 'E', 'test_col'),
 ('C', 'D', 'E', 'test_col'),
 ('A', 'B', 'C', 'D', 'E'),
 ('A', 'B', 'C', 'D', 'test_col'),
 ('A', 'B', 'C', 'E', 'test_col'),
 ('A', 'B', 'D', 'E', 'test_col'),
 ('A', 'C', 'D', 'E', 'test_col'),
 ('B', 'C', 'D', 'E', 'test_col'),
 ('A', '